<a href="https://colab.research.google.com/github/nicolay-r/arekit-ss/blob/master/arekit_googletrans_sampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# select python version
!sudo update-alternatives --config python3
# check python version
!python --version
# install pip for new python
!sudo apt-get install python3.8-distutils
!wget https://bootstrap.pypa.io/pip/get-pip.py
!sudo python get-pip.py

There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.10   2         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.8    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in manual mode
Python 3.8.10
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.8-distutils'
python3-distutils is already the newest version (3.8.10-0ubuntu1~20.04).
python3-distutils set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
--2023-07-15 10:26:55--  https://bootstrap.pypa.io/pip/get-pip.py
Resolving bootstrap.pypa.io (bo

In [ ]:
!pip install git+https://github.com/nicolay-r/arekit-ss.git@master

In [ ]:
!pip install pandas     # is temporary due the the https://github.com/nicolay-r/AREkit/issues/494 issue

In [34]:
!python -m arekit.download_data

100% 53.5k/53.5k [00:00<00:00, 1.11MB/s]
100% 705k/705k [00:00<00:00, 3.11MB/s]
100% 54.2k/54.2k [00:00<00:00, 5.61MB/s]
100% 57.4k/57.4k [00:00<00:00, 6.07MB/s]
100% 737/737 [00:00<00:00, 444kB/s]
100% 3.40M/3.40M [00:00<00:00, 47.6MB/s]
100% 3.46M/3.46M [00:00<00:00, 51.1MB/s]
100% 27.6M/27.6M [00:00<00:00, 53.6MB/s]
100% 146M/146M [00:01<00:00, 109MB/s]
100% 52.1M/52.1M [00:00<00:00, 151MB/s]
100% 321M/321M [00:02<00:00, 143MB/s]
63.3kB [00:00, 746kB/s]
63.1kB [00:00, 1.29MB/s]
63.4kB [00:00, 1.71MB/s]


# Sentiment Analysis examples section

1. Scenario with the bert-based formatter from `RuSentRel` collection with text translations into `en` language using Google Translate API.

In [ ]:
!python -m arekit_ss.sample --writer csv --source rusentrel --sampler nn \
  --dest_lang en --docs_limit 5 --text_parser lm --output_dir 'out_rusentrel_nn'

In [43]:
!python -m arekit_ss.sample --writer csv --source rusentrel --sampler bert \
  --dest_lang en --docs_limit 5 --text_parser lm --output_dir 'out_rusentrel_bert'

Loading RuSentRel Collection: 100%|█████████████████████████████████████████████| 1253/1253 [00:01<00:00, 834.92opins/s]
sample [DataType.Train]: 1412it [00:48, 29.36it/s, docs_seen=5, doc_now=13]


2. Prompting scenario for further application of the sampled data in LLM.
You can compose your personal prompt:
1. `text` -- original text of the sentence
2. `s_ind`, `t_ind` -- word indices in the original text
3. `label` -- sentiment label of the text.

In [ ]:
!python -m arekit_ss.sample --writer json --source rusentrel --sampler prompt \
  --prompt "For text: '{text}', the attitude between '{s_ind}','{t_ind}' is: '{label}'" \
  --dest_lang en --docs_limit 1 --text_parser lm --output_dir 'out_rusentrel_prompt'

Now we can take a look onto the results:

In [ ]:
from os.path import join
from arekit.contrib.utils.data.readers.csv_pd import PandasCsvReader

reader = PandasCsvReader()
storage = reader.read("out_rusentrel_prompt/sample-train-0.csv")
storage._df.head()